In [ ]:
#
# Copyright (C) 2021 by Katja Gilly <katya@umh.es>
#
# This code is licensed under a Creative Commons Attribution 4.0 International License. (see LICENSE.txt for details)
#
# General Description - this notebook plots several delays figures.
#
# It creates 7 types of output files per CloudSim simulation log that contain: 
#   - a multiple delays figure per simulation across simulation time
#   - average e2e delay (and confident intervals) figures grouped per car ranges distinguishing host types
#   - average e2e delay (and confident intervals) figures grouped per number of edge hosts distinguishing host types
#   - average access network delay (and confident intervals) figures grouped per car ranges distinguishing host types
#   - average access network delay (and confident intervals) figures grouped  per number of edge hosts distinguishing host types
#   - average edge network delay (and confident intervals) figures grouped per car ranges distinguishing host types
#   - average edge network delay (and confident intervals) figures grouped per number of edge hosts  distinguishing host types
#
# The notebook can be easily changed to compute medians and 25%, 75% percentiles following the comments 

import numpy as np, scipy.stats as st
#
def compute_mean (dataDelay, vms, file_name, path):

  i = 0
  meanDelay = np.zeros((vms, 3))  # columns: acess, network, e2e
  while (i < vms):
    oneCarData = dataDelay[dataDelay.car.eq(i)].to_numpy()
    meanDelay[i] = np.mean(oneCarData [:,2]),  np.mean(oneCarData[:,3]), np.mean(oneCarData [:,4])
    i+=1

  x = pd.Series(range(0,vms))
  plt.clf()
  plt.figure()
  plt.plot(x, meanDelay[:,2], label="total delay")
  plt.plot(x, meanDelay[:,0], label="access delay")
  plt.plot(x, meanDelay[:,1], label="network delay")
  plt.xlabel("car number")
  plt.ylabel("delay (ms)")
  plt.title("Average delays per car")
  plt.legend()
  plt.savefig(path + "delays_" + file_name + '.png',dpi=300)

  # Compute global mean and confidence internvals
  globalMeanDelay = np.zeros((3, 3))  # columns: acess, network, e2e
  conf_intervals = np.zeros((2, 3))  # columns: acess, network, e2e
  globalMeanDelay[0,:] = np.nanmean(meanDelay [:,0]),  np.nanmean(meanDelay[:,1]),np.nanmean(meanDelay [:,2])
  confidence=0.95
  conf_intervals = st.t.interval(confidence, vms-1, loc=globalMeanDelay, scale=st.sem(meanDelay,nan_policy='omit'))
  globalMeanDelay[1,:] = conf_intervals[0][0] # upper confidence interval
  globalMeanDelay[2,:] = conf_intervals[1][0] # lower confidence interval
  # To compute media and percentiles instead of mean values, uncomment the following lines
  # globalMeanDelay[0,:] = np.nanmedian(meanDelay [:,0]),  np.nanmedian(meanDelay[:,1]),np.nanmedian(meanDelay [:,2])
  # globalMeanDelay[1,:] = np.nanpercentile(meanDelay [:,0], 75, axis=0), np.nanpercentile(meanDelay [:,1], 75, axis=0), np.nanpercentile(meanDelay [:,2], 75, axis=0) 
  # globalMeanDelay[2,:] = np.nanpercentile(meanDelay [:,0], 25, axis=0), np.nanpercentile(meanDelay [:,1], 25, axis=0), np.nanpercentile(meanDelay [:,2], 25, axis=0) 
  print(globalMeanDelay) 

  return globalMeanDelay

time: 43.4 ms (started: 2021-09-01 09:30:00 +00:00)


In [ ]:
import numpy as np, scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt

def plot_figures(avgDelay_e2e, lower_confInt_e2e, upper_confInf_e2e, path_images, scenarios, edge_hosts, car_ranges):
  
  # 1st dim: vehicles 
  # 2nd dim: delays (acess, network, e2e)
  # 3rd dim: hosts and type (5x2)
  # group results
  avgDelay_grouped = np.zeros((4,3,10))
  lower_confInt_grouped = np.zeros((4,3,10))
  upper_confInt_grouped = np.zeros((4,3,10))

  seeds = 2 # (0,1,2)
  start = np.arange(0, 12, 3).tolist() 

  for i in range(4): # different seeds   
    for scen in range(10): # different scenarios 
      avgDelay_grouped[i,0,scen] = np.mean(avgDelay_e2e[start[i]:(start[i]+seeds+1),0,scen])
      avgDelay_grouped[i,1,scen] = np.mean(avgDelay_e2e[start[i]:(start[i]+seeds+1),1,scen])
      avgDelay_grouped[i,2,scen] = np.mean(avgDelay_e2e[start[i]:(start[i]+seeds+1),2,scen])

      lower_confInt_grouped[i,0,scen] = np.mean(lower_confInt_e2e[start[i]:(start[i]+seeds+1),0,scen])
      lower_confInt_grouped[i,1,scen] = np.mean(lower_confInt_e2e[start[i]:(start[i]+seeds+1),1,scen])
      lower_confInt_grouped[i,2,scen] = np.mean(lower_confInt_e2e[start[i]:(start[i]+seeds+1),2,scen])

      upper_confInt_grouped[i,0,scen] = np.mean(upper_confInf_e2e[start[i]:(start[i]+seeds+1),0,scen])
      upper_confInt_grouped[i,1,scen] = np.mean(upper_confInf_e2e[start[i]:(start[i]+seeds+1),1,scen])
      upper_confInt_grouped[i,2,scen] = np.mean(upper_confInf_e2e[start[i]:(start[i]+seeds+1),2,scen])

      
  # END-TO-END DELAY FIGURES --------------------------------------------------------------------------

  # Figure: Average e2e delay grouped per car ranges - host type 1
  # x axis: "49xx","57xx","69xx","85xx"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for scen in range(5): # different scenarios 
    ax.plot(car_ranges, avgDelay_grouped[:,2,scen], label=scenarios[scen][0])
    ax.fill_between(range(4), lower_confInt_grouped[:,2,scen], upper_confInt_grouped[:,2,scen], alpha=0.2)
    plt.xlabel("number of cars")
    plt.ylabel("delay (ms)")
    plt.title("Average end to end delay grouped per car ranges - type 1")
    plt.legend()
    plt.savefig(path_images + "e2e_delays_per_car_ranges-host_1.png",dpi=300)

  # Figure: Average e2e delay grouped per car ranges - host type 0
  # x axis: "49xx","57xx","69xx","85xx"
  # y axis: average delay + Cis  
  plt.figure(dpi=300)
  fig, ax = plt.subplots()
  for scen in range(5): # different scenarios 
    ax.plot(car_ranges, avgDelay_grouped[:,2,scen+5], label=scenarios[scen][0])
    ax.fill_between(range(4), lower_confInt_grouped[:,2,scen+5], upper_confInt_grouped[:,2,scen+5], alpha=0.2)
    plt.xlabel("number of cars")
    plt.ylabel("delay (ms)")
    plt.title("Average end to end delay grouped per car ranges - type 0")
    plt.legend()
    plt.savefig(path_images + "e2e_delays_per_car_ranges-host_0.png",dpi=300)

  # Figure: Average e2e delay grouped per edge hosts - host type 1
  # x axis: "45","63","81","99","117"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for car in range(4): # different car ranges 
    ax.plot(edge_hosts, avgDelay_grouped[car,2,0:5], label=car_ranges[car])
    ax.fill_between(range(5), lower_confInt_grouped[car,2,0:5], upper_confInt_grouped[car,2,0:5], alpha=0.2)
    plt.xlabel("number of edge hosts")
    plt.ylabel("delay (ms)")
    plt.title("Average end to end delay grouped per edge hosts - type 1")
    plt.legend()
    plt.savefig(path_images + "e2e_delays_per_host_ranges-host_1.png",dpi=300)

  # Figure: Average e2e delay grouped per edge hosts - host type 0
  # x axis: "45","63","81","99","117"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for car in range(4): # different car ranges 
    ax.plot(edge_hosts, avgDelay_grouped[car,2,5:10], label=car_ranges[car])
    ax.fill_between(range(5), lower_confInt_grouped[car,2,5:10], upper_confInt_grouped[car,2,5:10], alpha=0.2)
    plt.xlabel("number of edge hosts")
    plt.ylabel("delay (ms)")
    plt.title("Average end to end delay grouped per edge hosts - type 0")
    plt.legend()
    plt.savefig(path_images + "e2e_delays_per_host_ranges-host_0.png",dpi=300)


  # ACCESS NETWORK DELAY FIGURES --------------------------------------------------------------------------

  # Figure: Average access network delay grouped per car ranges - host type 1
  # x axis: "49xx","57xx","69xx","85xx"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for scen in range(5): # different scenarios 
    ax.plot(car_ranges, avgDelay_grouped[:,0,scen], label=scenarios[scen])
    ax.fill_between(range(4), lower_confInt_grouped[:,0,scen], upper_confInt_grouped[:,0,scen], alpha=0.2)
    plt.xlabel("number of cars")
    plt.ylabel("delay (ms)")
    plt.title("Average access network delay grouped per car ranges - host 1")
    plt.legend()
    plt.savefig(path_images + "access_delays_per_car_ranges-host_1.png",dpi=300)

  # Figure: Average access network delay grouped per car ranges - host type 0
  # x axis: "49xx","57xx","69xx","85xx"
  # y axis: average delay + Cis  
  plt.figure(dpi=300)
  fig, ax = plt.subplots()
  for scen in range(5): # different scenarios 
    ax.plot(car_ranges, avgDelay_grouped[:,0,scen+5], label=scenarios[scen])
    ax.fill_between(range(4), lower_confInt_grouped[:,0,scen+5], upper_confInt_grouped[:,0,scen+5], alpha=0.2)
    plt.xlabel("number of cars")
    plt.ylabel("delay (ms)")
    plt.title("Average access network delay grouped per car ranges - host 0")
    plt.legend()
    plt.savefig(path_images + "access_delays_per_car_ranges-host_0.png",dpi=300)

  # Figure: Average access network delay grouped per edge hosts - host type 1
  # x axis: "45","63","81","99","117"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for car in range(4): # different car ranges 
    ax.plot(edge_hosts, avgDelay_grouped[car,0,0:5], label=car_ranges[car])
    ax.fill_between(range(5), lower_confInt_grouped[car,0,0:5], upper_confInt_grouped[car,0,0:5], alpha=0.2)
    plt.xlabel("number of edge hosts")
    plt.ylabel("delay (ms)")
    plt.title("Average access network delay grouped grouped per edge hosts - host 1")
    plt.legend()
    plt.savefig(path_images + "access_delays_per_host_ranges-host_1.png",dpi=300)

  # Figure: Average access network delay grouped per edge hosts - host type 0
  # x axis: "45","63","81","99","117"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for car in range(4): # different car ranges 
    ax.plot(edge_hosts, avgDelay_grouped[car,0,5:10], label=car_ranges[car])
    ax.fill_between(range(5), lower_confInt_grouped[car,0,5:10], upper_confInt_grouped[car,0,5:10], alpha=0.2)
    plt.xlabel("number of edge hosts")
    plt.ylabel("delay (ms)")
    plt.title("Average access network delay grouped grouped per edge hosts - host 0")
    plt.legend()
    plt.savefig(path_images + "access_delays_per_host_ranges-host_0.png",dpi=300)


  # EDGE AND CLOUD NETWORK DELAY FIGURES --------------------------------------------------------------------------

  # Figure: Average edge network delay grouped per car ranges - host type 1
  # x axis: "49xx","57xx","69xx","85xx"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for scen in range(5): # different scenarios 
    ax.plot(car_ranges, avgDelay_grouped[:,1,scen], label=scenarios[scen])
    ax.fill_between(range(4), lower_confInt_grouped[:,1,scen], upper_confInt_grouped[:,1,scen], alpha=0.2)
    plt.xlabel("number of cars")
    plt.ylabel("delay (ms)")
    plt.title("Average edge network delay grouped per car ranges - host 1")
    plt.legend()
    plt.savefig(path_images + "edge_delays_per_car_ranges-host_1.png",dpi=300)

  # Figure: Average edge network delay grouped per car ranges - host type 0
  # x axis: "49xx","57xx","69xx","85xx"
  # y axis: average delay + Cis  
  plt.figure(dpi=300)
  fig, ax = plt.subplots()
  for scen in range(5): # different scenarios 
    ax.plot(car_ranges, avgDelay_grouped[:,1,scen+5], label=scenarios[scen])
    ax.fill_between(range(4), lower_confInt_grouped[:,1,scen+5], upper_confInt_grouped[:,1,scen+5], alpha=0.2)
    plt.xlabel("number of cars")
    plt.ylabel("delay (ms)")
    plt.title("Average edge network delay grouped per car ranges - host 0")
    plt.legend()
    plt.savefig(path_images + "edge_delays_per_car_ranges-host_0.png",dpi=300)

  # Figure: Average access network delay grouped per edge hosts - host type 1
  # x axis: "45","63","81","99","117"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for car in range(4): # different car ranges 
    ax.plot(edge_hosts, avgDelay_grouped[car,1,0:5], label=car_ranges[car])
    ax.fill_between(range(5), lower_confInt_grouped[car,1,0:5], upper_confInt_grouped[car,1,0:5], alpha=0.2)
    plt.xlabel("number of edge hosts")
    plt.ylabel("delay (ms)")
    plt.title("Average edge network delay grouped grouped per edge hosts - host 1")
    plt.legend()
    plt.savefig(path_images + "edge_delays_per_host_ranges-host_1.png",dpi=300)

  # Figure: Average access network delay grouped per edge hosts - host type 0
  # x axis: "45","63","81","99","117"
  # y axis: average delay + Cis     
  plt.figure(dpi=300) 
  fig, ax = plt.subplots()
  for car in range(4): # different car ranges 
    ax.plot(edge_hosts, avgDelay_grouped[car,1,5:10], label=car_ranges[car])
    ax.fill_between(range(5), lower_confInt_grouped[car,1,5:10], upper_confInt_grouped[car,1,5:10], alpha=0.2)
    plt.xlabel("number of edge hosts")
    plt.ylabel("delay (ms)")
    plt.title("Average edge network delay grouped grouped per edge hosts - host 0")
    plt.legend()
    plt.savefig(path_images + "edge_delays_per_host_ranges-host_0.png",dpi=300)

time: 639 ms (started: 2021-09-01 09:30:00 +00:00)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

path_Delays = "/content/drive/MyDrive/processed_logs/cloudsim_output/e2d_delays_3600s/"
path_images = "/content/drive/MyDrive/processed_logs/cloudsim_output/images/"

# Configure in case of directories in input files 
dirs = ['r2_e2eDelay_49xx','r2_e2eDelay_57xx','r2_e2eDelay_69xx','r2_e2eDelay_85xx']

# define vehicles and scenarios
scenarios = ["45 1 sc sc-mad","63 1 sc sc-mad","81 1 sc sc-mad","99 1 sc sc-mad","117 1 sc sc-mad",
             "45 0 sc sc-mad","63 0 sc sc-mad","81 0 sc sc-mad","99 0 sc sc-mad","117 0 sc sc-mad"]
# simulations of 3600s
vehicles = ['4928','4951','4955','5712','5734','5749','6900','6908','6923','8589','8595','8630']
VMs = ['1800','1805','1788','2072','2046','2089','2509','2493','2498','3184','3133','3185']
edge_hosts = ("45","63","81","99","117")
car_ranges = ("49xx","57xx","69xx","85xx")
duration="3600"
# Defining arrays:
# 1st dim: vehicles 
# 2nd dim: delays (access, network, e2e)
# 3rd dim: hosts and type (5x2)
avgDelay_e2e = np.zeros((12,3,10))
upper_confInf_e2e = np.zeros((12,3,10))
lower_confInt_e2e = np.zeros((12,3,10))

for scen in scenarios:
  for vehi in vehicles:
    veh_ind = vehicles.index(vehi)
    sce_ind = scenarios.index(scen)

    group = math.floor(veh_ind/3)  # group of seeds of the same scenario
    print("group:", group, " -> ", dirs[group])
    scen_split = scen.split()
    file_name  = "e2eDelay_" + scen_split[0] + "_" + scen_split[1] + "_sc_sc-mad_" + str(VMs[veh_ind]) + "_" + str(vehi) + ".csv"
    #dataDelay = pd.read_csv(path_Delays + dirs[group] + "/" + file_name, sep=',', header=0)
    dataDelay = pd.read_csv(path_Delays + file_name, sep=',', header=0)
    
    print("index vehicles:", veh_ind, " -> ", vehi)
    print("index scenarios", sce_ind%5, " -> ", scen)
  
    # write average delay in avgDelay_e2e, confidence intervals in lower_confInt_e2e and upper_confInf_e2e
    temp1 = compute_mean (dataDelay, int(VMs[veh_ind]), file_name, path_images) 
    #print("temp1:",temp1)
    avgDelay_e2e[veh_ind,:,sce_ind] = temp1[0]
    lower_confInt_e2e[veh_ind,:,sce_ind] = temp1[1]
    upper_confInf_e2e[veh_ind,:,sce_ind] = temp1[2]
#    
print(avgDelay_e2e)
plot_figures(avgDelay_e2e, lower_confInt_e2e, upper_confInf_e2e, path_images, scenarios, edge_hosts, car_ranges)